# Flight Delay Prediction ✈️🕗

This notebook contains the steps taken to get the data, explore and know about data and preprocess the data for better intrepretation for machine learning model.


## Step 0: Defining the problem statement

**Objective:**  
To determine whether an airplane is going to get delayed or not, if yes, find out by the delay (in minutes).  

**Speculated Solution:**  
Make a model to classify whether a flight is going to get delayed or not, as well as prepare a regression model to predict the time of delay.

## Step 1: Getting the data

The data used for this problem is available on [Kaggle](https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018).



- Use Kaggle's API to download the data into the Colab Environment
- Get the utility functions that may help later.
- Configure data files to read using Python

The data contains multiple features for each year from 2009 to 2018.

Glossary of the features:

| Name      | Meaning |
| :----------------:        |    :-------------------:   |
| FL_Date      | Date of the Flight       |
| OP_CARRIER   | Airline Identifier        |
| OP_CARRIER_FL_NUM   | Flight Number        |
| ORIGIN   | Starting Airport Code        |
| DEST   | Destination Airport Code        |
| CRS_DEP_TIME   | Planned Departure Time        |
| DEP_TIME   | Actual Departure Time        |
| DEP_DELAY   | Total Delay on Departure in minutes        |
| TAXI_OUT    | Time duration elapsed between departure from the origin airport gate and wheels off        |
| WHEELS_OFF    | Time point that the aircraft's wheels leave the ground        |
| WHEELS_ON    | Time point that the aircraft's wheels touch on the ground        |
| TAXI_IN    | Time duration elapsed between wheels-on and gate arrival at the destination airport        |
| CRS_ARR_TIME    | Planned arrival time       |
| ARR_TIME     | Actual Arrival Time       |
| ARR_DELAY     | Total Delay on Arrival in minutes       |
| CANCELLED     | Flight Cancelled (1 = cancelled)       |
| CANCELLATION_CODE     | Reason for Cancellation of flight( `A - Airline/Carrier; B - Weather; C - National Air System; D - Security`)   |
| DIVERTED     | Aircraft landed on different airport that the one scheduled   |
| CRS_ELAPSED_TIME     | Planned time amount needed for the flight trip   |
| ACTUAL_ELAPSED_TIME     | `AIR_TIME`+ `TAXI_IN` + `TAXI_OUT`   |
| AIR_TIME     | The time duration between wheels_off and wheels_on time   |
| DISTANCE     | Distance between two airports   |
| CARRIER_DELAY     | Delay caused by the airline in minutes   |
| WEATHER_DELAY     | Delay caused by weather   |
| NAS_DELAY     | Delay caused by air system   |
| SECURITY_DELAY      | Delay caused by security reasons   |
| LATE_AIRCRAFT_DELAY      | Delay caused by security  |



In [1]:
# Getting the helper functions script
!wget https://raw.githubusercontent.com/ishandandekar/Airline-delay-prediction/main/src/utils/utils.py

# Install the kaggle library
!pip install -q kaggle

# Upload the Kaggle API keys
from google.colab import files
files.upload()

!mkdir ~/.kaggle

# Copy the json file to the folder
!cp kaggle.json ~/.kaggle

# Change permissions for keys to work with the Kaggle API
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018 --quiet

# Creating a directory to store all kinds of data
!mkdir data

# Unzip data
from utils import unzip_data
unzip_data('airline-delay-and-cancellation-data-2009-2018.zip', data_dir="data/raw")

--2022-12-23 11:50:47--  https://raw.githubusercontent.com/ishandandekar/Airline-delay-prediction/main/src/utils/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1838 (1.8K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   1.79K  --.-KB/s    in 0s      

2022-12-23 11:50:47 (27.1 MB/s) - ‘utils.py’ saved [1838/1838]



Saving kaggle.json to kaggle.json


## Step 2: Know more about the data
- Load in the data using Pandas
- Optimize data for faster reading
- Get the statistics about the data
- Fix missing/incorrect values
- Analyze features
- Summarize observations

In [ ]:
from glob import glob
from datetime import datetime

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def validate_int2str(col):
    try:
        if col: 
            col = int(float(col))         
            if (col):
                col = str(col).zfill(4) 
                col = datetime.strptime(col, '%H%M').time().strftime("%I:%M") 
            return col   
        else: 
            return np.NaN          
    except Exception as e:      
        return np.NaN

def optimize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
  df = df.copy(deep=True)

  print(f"Before memory optimization: {df.memory_usage(deep=True).sum() * 0.000001} MBs")

  print("Dropping columns...")
  df = df.drop('Unnamed: 27', axis=1)
  # df = df.drop('CARRIER_DELAY', axis=1)
  # df = df.drop('WEATHER_DELAY', axis=1)
  # df = df.drop('NAS_DELAY', axis=1)
  # df = df.drop('SECURITY_DELAY', axis=1)
  # df = df.drop('CANCELLATION_CODE', axis=1)

  print("Changing data types...")
  df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], yearfirst=True)
  df['DEP_TIME'] = df['DEP_TIME'].astype('int8', errors='ignore')
  df['DEP_DELAY'] = df['DEP_DELAY'].astype('int8', errors='ignore')
  df['OP_CARRIER_FL_NUM'] = df['OP_CARRIER_FL_NUM'].astype('category', errors='ignore')
  df['OP_CARRIER'] = df['OP_CARRIER'].astype('category', errors='ignore')
  df['ORIGIN'] = df['ORIGIN'].astype('category', errors='ignore')
  df['DEST'] = df['DEST'].astype('category', errors='ignore')
  df['CANCELLED'] = df['CANCELLED'].astype('bool', errors='ignore')
  df['DIVERTED'] = df['DIVERTED'].astype('bool', errors='ignore')
  df['CANCELLATION_CODE'] = df['CANCELLATION_CODE'].astype('category', errors='ignore')

  print("Parsing time columns...")
  cols_ = ['CRS_DEP_TIME', 'DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_TIME', 'NAS_DELAY', 'SECURITY_DELAY', 'CARRIER_DELAY', 'LATE_AIRCRAFT_DELAY', 'WHEELS_OFF', 'WHEELS_ON']
  for col_ in cols_:
    df[col_] = df[col_].apply(lambda x: validate_int2str(x))

  # df['CRS_DEP_TIME'] = df['CRS_DEP_TIME']
  # df['DEP_TIME'] = df['DEP_TIME'].apply(lambda x: validate_int2str(x))
  # df['CRS_ARR_TIME'] = df['CRS_ARR_TIME'].apply(lambda x: validate_int2str(x))
  # df['ARR_TIME'] = df['ARR_TIME'].apply(lambda x: validate_int2str(x))
  # df['NAS_DELAY'] = df['NAS_DELAY'].apply(lambda x: validate_int2str(x))
  # df['SECURITY_DELAY'] = df['SECURITY_DELAY'].apply(lambda x: validate_int2str(x))
  # df['CARRIER_DELAY'] = df['CARRIER_DELAY'].apply(lambda x: validate_int2str(x))
  # df['LATE_AIRCRAFT_DELAY'] = df['LATE_AIRCRAFT_DELAY'].apply(lambda x: validate_int2str(x))
  # df['WHEELS_OFF'] = df['WHEELS_OFF'].apply(lambda x: validate_int2str(x))
  # df['WHEELS_ON'] = df['WHEELS_ON'].apply(lambda x: validate_int2str(x))

  print(f"After memory optimization: {df.memory_usage(deep=True).sum() * 0.000001} MBs")

  return df

# csv_files = glob("data/raw/*.csv")
# dfs = []

# for f in tqdm(csv_files):
#   df = pd.read_csv(f)
#   df = optimize_dataframe(df)
#   dfs.append(df)

# df = pd.concat(dfs, ignore_index=True)
# print(f"Shape of the data: {df.shape}")

In [ ]:
# Checking the first 5 rows of data


----
TEST

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df2018_chunks = pd.read_csv('/content/data/raw/2018.csv')

In [3]:
df2018_chunks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   OP_CARRIER_FL_NUM    int64  
 3   ORIGIN               object 
 4   DEST                 object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  CANCELLED            float64
 16  CANCELLATION_CODE    object 
 17  DIVERTED             float64
 18  CRS_ELAPSED_TIME     float64
 19  ACTUAL_ELAPSED_TIME  float64
 20  AIR_TIME             float64
 21  DISTANCE             float64
 22  CARRIER_DELAY        float64
 23  WEATHER_DELAY        float64
 24

In [4]:
df2018_chunks['OP_CARRIER'].unique()

array(['UA', 'AS', '9E', 'B6', 'EV', 'F9', 'G4', 'HA', 'MQ', 'NK', 'OH',
       'OO', 'VX', 'WN', 'YV', 'YX', 'AA', 'DL'], dtype=object)

SOURCE : https://www.wikiwand.com/en/List_of_airlines_of_the_United_States

In [5]:
df2018_chunks['OP_CARRIER'] = df2018_chunks['OP_CARRIER'].map({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
})

In [7]:
df2018_chunks.OP_CARRIER.nunique()

18

In [8]:
df2018_chunks = df2018_chunks.drop(["Unnamed: 27"], axis=1)

DROPPING CANCELLED FLIGHTS BECAUSE WE ONLY WANT THE ONES WHICH GOT DELAYED

In [9]:
df2018_chunks = df2018_chunks[(df2018_chunks['CANCELLED'] == 0)]
df2018_chunks = df2018_chunks.drop(['CANCELLED'], axis = 1)

REMOVING CANCELLATION_CODE BECAUSE OF NA VALUES

In [12]:
df2018_chunks = df2018_chunks.drop(["CANCELLATION_CODE"], axis=1)

IT IS THE RESPONSIBILITY OF THE DIVERTED FLIGHTS TO TRANSPORT THE PASSENGERS TO THE CORRECT DESTINATION, SO THIS WILL BE CONSIDERED AS DELAYED.

In [13]:
df2018_chunks.DIVERTED.value_counts()

0.0    7079005
1.0      17857
Name: DIVERTED, dtype: int64

<!-- DIVERTED 1 means the flight has been diverted   -->
DROPPING DIVERTED COLUMN BECAUSE NO INFORMATION

DEALING WITH DELAY REASONS

In [14]:
print(df2018_chunks.CARRIER_DELAY.isna().sum())
print(df2018_chunks.WEATHER_DELAY.isna().sum())
print(df2018_chunks.NAS_DELAY.isna().sum())
print(df2018_chunks.SECURITY_DELAY.isna().sum())
print(df2018_chunks.LATE_AIRCRAFT_DELAY.isna().sum())

5744152
5744152
5744152
5744152
5744152


In [15]:
print("Percentage of valid data:", 100 - (df2018_chunks.CARRIER_DELAY.isna().sum()*100/len(df2018_chunks)))
print("Percentage of missing values:", (df2018_chunks.CARRIER_DELAY.isna().sum()*100/len(df2018_chunks)))

Percentage of valid data: 19.060677803795542
Percentage of missing values: 80.93932219620446


In [16]:
df2018_chunks = df2018_chunks.drop(['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], axis=1)
print('The current shape of df1 is:', df2018_chunks.shape)

The current shape of df1 is: (7096862, 20)


DROPPING IRRELAVENT COLUMN

In [17]:
df2018_chunks = df2018_chunks.drop(['OP_CARRIER_FL_NUM'], axis=1)

BELOW COLUMNS DO NOT HELP TOO, WE ALREADY HAVE DEP_DELAY

In [18]:
df2018_chunks = df2018_chunks.drop(columns=['DEP_TIME', 'ARR_TIME'])

In [19]:
df2018_chunks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7096862 entries, 0 to 7213445
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   ORIGIN               object 
 3   DEST                 object 
 4   CRS_DEP_TIME         int64  
 5   DEP_DELAY            float64
 6   TAXI_OUT             float64
 7   WHEELS_OFF           float64
 8   WHEELS_ON            float64
 9   TAXI_IN              float64
 10  CRS_ARR_TIME         int64  
 11  ARR_DELAY            float64
 12  DIVERTED             float64
 13  CRS_ELAPSED_TIME     float64
 14  ACTUAL_ELAPSED_TIME  float64
 15  AIR_TIME             float64
 16  DISTANCE             float64
dtypes: float64(11), int64(2), object(4)
memory usage: 974.6+ MB


HANDLING MISSING VALUES

In [20]:
print('Number of total missing values:', df2018_chunks.isna().sum().sum())
print('df1 shape:', df2018_chunks.shape)

Number of total missing values: 66246
df1 shape: (7096862, 17)


FILLING `DEP_DELAY` WITH 0 BECAUSE THE VALUE WILL ONLY BE NaN, ONLY IF THE FLIGHT WASN'T DELAYED

In [21]:
df2018_chunks["DEP_DELAY"] = df2018_chunks["DEP_DELAY"].fillna(0)
df2018_chunks.isna().sum()

FL_DATE                    0
OP_CARRIER                 0
ORIGIN                     0
DEST                       0
CRS_DEP_TIME               0
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON               2662
TAXI_IN                 2662
CRS_ARR_TIME               0
ARR_DELAY              20456
DIVERTED                   0
CRS_ELAPSED_TIME           7
ACTUAL_ELAPSED_TIME    17858
AIR_TIME               17858
DISTANCE                   0
dtype: int64

FILLING THIS WITH MEAN BECUASE NO WAY TO CALCULATE THE ACTUAL VALUES

In [22]:
df2018_chunks['TAXI_IN'].fillna((df2018_chunks['TAXI_IN'].mean()), inplace=True)
df2018_chunks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7096862 entries, 0 to 7213445
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   ORIGIN               object 
 3   DEST                 object 
 4   CRS_DEP_TIME         int64  
 5   DEP_DELAY            float64
 6   TAXI_OUT             float64
 7   WHEELS_OFF           float64
 8   WHEELS_ON            float64
 9   TAXI_IN              float64
 10  CRS_ARR_TIME         int64  
 11  ARR_DELAY            float64
 12  DIVERTED             float64
 13  CRS_ELAPSED_TIME     float64
 14  ACTUAL_ELAPSED_TIME  float64
 15  AIR_TIME             float64
 16  DISTANCE             float64
dtypes: float64(11), int64(2), object(4)
memory usage: 974.6+ MB


DROPPING REST OF THE ROWS

In [24]:
df1 = df2018_chunks.dropna()

DEALING WITH TIME RELATED COLUMNS

In [25]:
df1['CRS_DEP_TIME'] = np.ceil(df1['CRS_DEP_TIME']/600).apply(int)
df1['WHEELS_OFF'] = np.ceil(df1['WHEELS_OFF']/600).apply(int) 
df1['WHEELS_ON'] = np.ceil(df1['WHEELS_ON']/600).apply(int)
df1['CRS_ARR_TIME'] = np.ceil(df1['CRS_ARR_TIME']/600).apply(int)

In [26]:
df1['DAY'] = pd.DatetimeIndex(df1['FL_DATE']).day
df1['MONTH'] = pd.DatetimeIndex(df1['FL_DATE']).month

COLUMN FOR BINARY CLASSIFICATION

In [27]:
status = []

for value in df1['ARR_DELAY']:
    if value < 0:
        status.append(0)
    else:
        status.append(1)
df1['FLIGHT_STATUS'] = status

---

Per Wikipedia: https://www.wikiwand.com/en/Flight_cancellation_and_delay

```
Delays are divided into three categories, namely "on time or small delay" (up to 15 minutes delay), "Medium delay" (15 – 45 minutes delay) and "Large delay" ( 45 minutes delay).
```